## **Importing Dataset**

In [3]:
import kagglehub

path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-and-cat-classification-dataset


## **Question 1**

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Function to extract grayscale image features and flatten them
def load_image_data(image_dir, label, image_size=(64, 64)):
    data, labels = [], []
    for file in os.listdir(image_dir):
        if file.endswith('.jpg'):
            image_path = os.path.join(image_dir, file)
            try:
                img = Image.open(image_path).convert('L')  # Convert to grayscale
                img = img.resize(image_size)
                data.append(np.array(img).flatten())      # Flatten the image
                labels.append(label)
            except Exception as e:
                pass  # Skip corrupted or unreadable images
    return np.array(data), np.array(labels)

# Set dataset directories
cat_path = "/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat"
dog_path = "/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog"

# Load and label the data
cat_data, cat_labels = load_image_data(cat_path, label=0)
dog_data, dog_labels = load_image_data(dog_path, label=1)

# Combine cat and dog data
X = np.vstack((cat_data, dog_data))
y = np.concatenate((cat_labels, dog_labels))

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train a feed-forward neural network
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=100, random_state=1)
mlp.fit(X_train, y_train)

# Predict and evaluate model performance
y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


              precision    recall  f1-score   support

           0       0.59      0.33      0.42      2515
           1       0.53      0.77      0.63      2485

    accuracy                           0.55      5000
   macro avg       0.56      0.55      0.53      5000
weighted avg       0.56      0.55      0.53      5000



**Question:** What were the limitations of FFNNs with high-dimensional image inputs?

**Answer:**
Feedforward Neural Networks (FFNNs) have several limitations when applied to high-dimensional image inputs. One of the primary issues is the large number of parameters involved. Since each pixel in an image becomes an input feature, even a relatively small grayscale image of size 64×64 results in 4,096 input nodes. When these are connected to hidden layers in a fully connected manner, the number of weights increases exponentially, making the model computationally expensive and highly prone to overfitting, especially when the available training data is limited. Additionally, FFNNs lack spatial awareness—they treat each pixel as an independent feature and do not account for local patterns or the spatial relationships between pixels. This means they are unable to capture important visual features like edges, textures, or shapes that are critical for understanding image content. Moreover, FFNNs do not have the property of translation invariance; they cannot recognize objects in varying positions across the image, which is essential in image classification tasks. They also do not utilize parameter sharing, which leads to redundant computations and inefficient use of model capacity. Because of these limitations, FFNNs generally perform poorly on complex visual tasks compared to Convolutional Neural Networks (CNNs), which are specifically designed to handle image data by exploiting spatial hierarchies, local connectivity, and weight sharing. Therefore, while FFNNs may still be useful for simple or low-dimensional image tasks, CNNs are far more suitable and effective for real-world image processing applications.


## **Question 2**

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the dataset directory
dataset_dir = "/kaggle/input/dog-and-cat-classification-dataset/PetImages"

# Initialize the image data generator with rescaling and validation split
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create the training data generator
train_data = image_generator.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Create the validation data generator
validation_data = image_generator.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define a basic Feedforward Neural Network (BPNN) model
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),      # Flatten the input image
    Dense(128, activation='relu'),         # First hidden layer
    Dense(64, activation='relu'),          # Second hidden layer
    Dense(1, activation='sigmoid')         # Output layer for binary classification
])

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model architecture
model.summary()

# Train the model using the training and validation data
model.fit(train_data, epochs=10, validation_data=validation_data)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,572,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,581,313 (6.03 MB)

...

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 34/625 ━━━━━━━━━━━━━━━━━━━━ 36s 62ms/step - accuracy: 0.5717 - loss: 1.4449

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 75ms/step - accuracy: 0.5483 - loss: 0.8527 - val_accuracy: 0.5348 - val_loss: 0.6945
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 73ms/step - accuracy: 0.5827 - loss: 0.6755 - val_accuracy: 0.6066 - val_loss: 0.6582
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 47s 76ms/step - accuracy: 0.5852 - loss: 0.6672 - val_accuracy: 0.6116 - val_loss: 0.6539
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 47s 75ms/step - accuracy: 0.6106 - loss: 0.6547 - val_accuracy: 0.5830 - val_loss: 0.6803
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 73ms/step - accuracy: 0.6128 - loss: 0.6514 - val_accuracy: 0.5878 - val_loss: 0.6600
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 72ms/step - accuracy: 0.6076 - loss: 0.6504 - val_accuracy: 0.6194 - val_loss: 0.6514
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 47s 76ms/step - accuracy: 0.6352 - loss: 0.6366 - val_accuracy: 0.6094 - val_loss: 0.6526
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 74ms/step - accuracy: 0.6378 - loss: 0.6287 - val_accurac